# Ensemble Learning and Random Forest

Lets say you give a complex question to thousands of random people, then aggregate their answers. In many cases you will find that this answer would be better than an experts answer. This is called "wisdom of the crowd". This is the concept upon which Ensemble Learning is built. For ensemble learning, we aggregate the predictions of a group of predictors (eg, classifiers or regressors), this aggregated prediction will often give a better result than that of the best individual predictor. A group of predictors is called an **Ensemble**, this technique is called **Ensemble Learning** and a Ensemble Learning algorithm is called **Ensemble method**

We saw this method in action in the chapter on Decision Trees. In the assignment section we built our own implementation of a Random Forest Classifier. We had trained a group of Decision Tree Classifiers on subsets of the training data, we then made predictions on the test data, and we aggregated by taking the prediction that was the most frequent. Such an ensemble of Decision Trees is called a **Random Forest**. Despite the simplicity of such a method, this algorithm is one of the most powerful machine learning algorithms. In this section we will be discussing the most popular ensemble methods in detail such as bagging, boosting and stacking. We will also discuss the Random Forest Algorithm in greater detail.

This chapter can be divided into the following sections:
1. Voting Classifiers
2. Bagging and Pasting
3. Random Patches and Random Subspaces
4. Random Forests
5. Boosting
6. Stacking

## 1. Voting Classifiers

Lets take a scenario where you have trained a few classifiers, each achieving about 80% accuracy. For example, they could be a LogisticRegression classifier, an SVM classifier, a RandomForest classifier, a K-Nearest Neighbors classifier and perhaps a few more. 

<center>
<br>
<img src="https://miro.medium.com/max/962/1*Z6M1YaQ2k-0clMBpLEruOg.png" height="250">
<br>
</center>

We could create an even better classifier by aggregating these classifiers together to create an ensemble. We would do this by aggregating the predictions of the classifiers and predict the class that gets the most votes. This type of aggregation (majority vote classifier) is called a **hard voting classifer**.

<center>
<br>
<img src="https://cdn-images-1.medium.com/max/1000/0*c0Eg6-UArkslgviw.png" height="300">
<br>
</center>

A soft voting classifier would on the other hand predict the probability that a particular instance belongs to a particular class and then predict the class with the highest probability. This is called a **soft voting classifier**. (This is possible only if all the classifiers can estimate class probabilities, e.g. the LogisticRegressor that has a predict_proba() method).

This voting classifer often achieves a higher accuracy than the best classifier in the ensemble. This is true even if the ensemble is made up of classifiers that are weak learners (in the sense that they do only slightly better than random guessing (e.g. a classifier that has an accuracy of 51%)). The ensemble can still be a strong learner provided there are a succificient number of weak learners and they are sufficiently diverse.

To understand how this is possible, we will use the example of a biased coin flip. Suppose you have a slightly biased coin that has a 51% chance of coming up heads and 49% of coming up tails. If this biased coin is tossed 1000 times, it will come up heads 510 times and tails 490 times, and hence a majority of heads. If you do the math, you will find that the probability of obtaining a majority of heads afer 1,000 tosses is close to 75%. The more you toss the coin, the higher is the probability (10,000 tosses, probability goes up to 97%). This is due to the **law of large numbers**, as you keep tossing the coin the ratio of heads to tails gets closer and closer to the probability of heads (51%). The below image shows that as the number of tosses increases the ratio of heads gets coser to 51%. By the end it is consistently above 50%.

<center>
<br>
<img src="https://miro.medium.com/max/1400/1*t5hIwNudkP1dQ5yk3NQBYw@2x.png" height="300">
<br>
</center>

For example if you build an ensemble containing 1000 classifiers that are individually correct only 51% of the time (barely better than random guessing), and you predict the majority voted class you might end up getting a 75% accuracy. This however is only true if all the classifiers are perfectly independent making uncorrelated errors, which will not be the case as it is trained on the same data. THey are likely to make the same types of errors, so there will be many majority votes for the wrong class, reducing the ensemble's accuracy.

Note: Ensemble methods work best when the predictors are as independent from one another as possible. One way to get the predictors to be diverse is to train them using very different algorithms, This increases the chance that they will make very different types of errors, improving the ensemble's accuracy. We can try to use this method on our Titanic assignment.

Given below is an implementation that trains a voting classifier in sklearn, composed of three diverse classifiers. They will be trained on the moons dataset. Earlier in the chapter on Decision Trees w had made our own implementation of a voting classifier. We will now use the sklearn implementation.

In [8]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

X, y = make_moons(n_samples=1000, noise=0.4, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

# voting hard as we are using majority vote
voting_clf = VotingClassifier(estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)], voting='hard')

voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [10]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_preds = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_preds))

LogisticRegression 0.836
RandomForestClassifier 0.868
SVC 0.86
VotingClassifier 0.868


Remember the RandomForestClassifier, is itself an ensemble, it is an ensemble of DecisionTreeClassifiers. 

If all classifiers are able to estimate probabilities, then you can tell sklearn to predict the class with the highest class probability averaged over all the individual classifiers. This is called **soft voting**. It often achieves higher performance than hard voting, this is beause it gived more weight to hgihly confident votes. To use soft voting, you simply need to switch the value of voting="hard" to voting="soft" and ensure that all classifiers can estimate class probabilites. This will not be the case for a default SVC, so you will need to set its probability hyperparameter to True (this will make the SVC class estimate class probabilities, slowing down training, and it will add a predict_proba() method).

Lets do this by implementing it in the following manner.

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC(probability=True)

# voting hard as we are using majority vote
voting_clf = VotingClassifier(estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)], voting='hard')

voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()),
                             ('svc', SVC(probability=True))])

In [12]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_preds = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_preds))

LogisticRegression 0.836
RandomForestClassifier 0.864
SVC 0.86
VotingClassifier 0.872


As you can see the performance of the ensemble has slightly improved simply by using soft voting.

## 2. Bagging and Pasting

We have already used the method of getting a diverse set by training different algorithms. However there is another approach where the algorithm is the same but the training data is different, i.e. we use the same training algorithm for every predictor, but we train them on different random subsets of the training data. When sampling is performed with replacement, this method is called **bagging** (also called bootstrap aggregating, used by Random Forests). When sampling is performing without replacement, it is called **pasting**.

Both bagging and sampling allow training instances to be sampled several times across multiple predictors, but only bagging allows training instances to be sampled several times for the same predictor.

<center>
<br>
<img src="https://raw.githubusercontent.com/Akramz/Hands-on-Machine-Learning-with-Scikit-Learn-Keras-and-TensorFlow/63b8d7a91ff1ca2fdc35947a6a390c5a81085bd2//static/imgs/bagging.png" height="300">
<br>
</center>

Once all the predictors are trained the ensemble can make a prediction for a new instance simply by aggregating the predictions of all predictors. The aggregation function is usually the *statistical mode* (we had used Scipy mode() function) for classification or the *average* for regression. Each individual predictor has a higher bias than if it were trained on the original training set, but aggregation reduces both bias and variance. The net result is that the ensemble will have a similar bias but lower variance, than a single predictor trained on the original training set.

Predictors can be trained in parallel, via different CPU cores or even different servers. Similarly predictions can be made in parallel. This is a major reason as to why bagging and pasting is so popular, they scale well.

This section will be divided into the following subsections: -
1. Bagging and Pasting in Scikit-Learn
2. Out of Bag Evaluation
3. Random Patches and Random Subspaces

### 2.1 Bagging and Pasting in Scikit-Learn

Sklearn offers a simple API for both bagging and pasting with the BaggingClassifier/BaggingRegrros. The following code trains an ensemble of 500 DecisionTree classifiers each is trained on 100 training instances randomly sampled from our training set with replacement (if bootstap=True this is Bagging, if you want Pasting then set value as False). The n_jobs parameter tells sklearn the number of CPU cores to use, this allows it to run in parallel. A value of -1 tells it to use all available cores. The implementation is as follows:

In [13]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500, max_samples=100, bootstrap=True, n_jobs =-1)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.856

The BaggingClassifier automatically chooses whether to perform soft or hard voting based on whether ot not the predictor given has the capability of estimating probabilites. Since this is a functionality of DecisionTreeClassifier, it performs soft voting.

<center>
<br>
<img src="https://raw.githubusercontent.com/Akramz/Hands-on-Machine-Learning-with-Scikit-Learn-Keras-and-TensorFlow/63b8d7a91ff1ca2fdc35947a6a390c5a81085bd2//static/imgs/dt_bagging.png" height="300">
<br>
</center>

Bootstrapping introduces a bit more diversity in he subsets that each predictor is trained on so bagging ends up with a slightly higher bias as compared to pasting but the extra diversity also means that the predictors end up being less correated, so the ensembles variance is reduce. overall bagging often results in better models, which is why it is generally preffered. however if you have spare time and CPU power you can use cross validation to evaluate both bagging and pasting and select the one that works best.

### 2.2 Out of Bag Evaluation

While bagging some instances may be sampled several times for any given predictor, while others may not be sampled at all. By default a BaggingClassifier samples m training instances with replacement (bootstrap=True), where m is the size of the training set. This means that only about 63% of the training instances are sampled on average for each predictor (as m grows the ratio appraches $1 - exp(-1) \approx 63.212%$). The reamining 37% of the training instances that are not sampled are called out of bag (oob) instances. It must be remembered that this is not the same for all predictors.

Since a predictor neever sees the oob instanced during training it can be evaluated on these instances without the need for a seperate validation set. You can evaluate the ensemble itself by averaging out the oob evaluations of each predictor. In sklearn you can set oob_score=True when creating a BaggingClassifier to request an automatic oob evaluation after training. The following code demonstrates this, the resulting evaluation score is available through the oob_score_ variable.

In [14]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500, 
    bootstrap=True, n_jobs=-1, oob_score=True)
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.8346666666666667

In [15]:
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.844

The oob decision function for each training instance is also available through the oob_decision_function_ variable (in this case since the base estimator has a predict_proba() method), the decision function returns the class proababilities for each training instance, For example the oob evaluation estimates that the first training instance ha a 68.25% probability of belonging to the positive class (and 31.75% of belonging to the negative class).

In [16]:
bag_clf.oob_decision_function_

array([[0.46111111, 0.53888889],
       [0.71111111, 0.28888889],
       [1.        , 0.        ],
       ...,
       [0.98387097, 0.01612903],
       [0.68181818, 0.31818182],
       [0.01197605, 0.98802395]])

### 2.3 Random Patches and Random Subspaces

The BaggingClassifier class supports sampling the features as well. Sampling is controlled by two hyperparameters: max_features and bootstrap features. They work the same way as max_samples and bootstrap, but for features sampling instead of instance sampling. Thus, each predictor will be trained on a random subset of the input features.

This technique is especially useful when we deal with high-dimensional inputs (such as images). Sampling both the training instances and features os ca;;ed tje Random Patches method. Keeping all training instances (bootstrap=False and max_samples=1.0) but sampling features (bootstrap_features = True and/or max_features to a value smaller than 1.0) is called the Random Subspaces method.

Sampling features results in even more predictor diversity, trading a bit more bias for a lower variance.


## 3. Random Forests

A Random Forest is an ensemble of Decision Trees, generally trained by the bagging method typically with max_samples set tot the size of the training set. instead of building a BaggingClassifier and passing it tot a DecisionTreeClassifier, you can use a RandomForestClassifier class, which is more convenient and optimised for Decision Trees (there is also a RandomForestRegressor class for regression tasks). The following code uses all available CPU core to train a Random Forest classifier  with 500 trees (each limited to maximum 16 nodes)

In [17]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

Apart from a few exceptions (splitter is absent (forced to random), presort is absent (forced to False), max_samples is absent (forced to 1.0), base_estimator is absent(forced to DecisoinTreeClassifier with the provided hyperparameters)) a RandomForestClassifier has all the hyperparameters of a DecisionTreeClassifier (to control how trees are grown) plus all the hyperparameters ofa BaggingClassifier to control the ensemble itself.

The Random Forest Algorithm introduces extra randomness when growing trees, instead of searching for the best feature when splitting a node, it searches for a best feature among a random subset of features. The algorithm results in greater tree diversity, which trades a hgiher bias for a lower variance, generally yielding an overall better model. The following BaggingClassifier is roughly equivalent to the previous RandomForestClassifier.

In [19]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(splitter="random", max_leaf_nodes=16), 
    n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1)

This section will be now be divided into the following subsections: -
1. Extra-Trees
2. Feature Importance

### 3.1 Extra-Trees

When you are growing a tree in a Random Forest, at each node only a random subset of the features is considered for splitting. It is possible to make trees even more random by using random thresholds for each feature rather than searching for the best possible thresholds. A forest of such extremely random trees is called an Extremely Randomized Trees ensemble (or Extra-Trees for short). Once again, this trades more bias for a lower variance. It also makes Extra-Trees much faster to train than regular Random Forests since finding the best possible threshold for each feature at every node is one of the most time-consuming tasks of growing a tree.

You can create an Extra-Trees classifier using sklearns ExtraTreesClassifier class. Its API is identical to the RandomForestClassifier class. Similarly the ExtraTreesRegressor class has the same API has the RandomForestRegressor class.

We cannot tell in advance whether a RandomForestClassifier will perform better ir worse than a ExtraTreeslassfier the only way to knwow is to try both and compare them using cross validation (tune the hyperparameters using grid search).

### 3.2 Feature Importance

Another great quality of Random Forests is that they can make it easy to measure the relatvie importance of each feature. Sklearn measure s afeatures importance by looking at how much the tree nodes that use that feature reduce impurity on average (by looking across all the trees in the forest), to be more precise it is a weighted avergae where each nodes weight is equal to thenumber of training samples that are associated with it. 

Sklearn comtes this score automatically for each feature after trainingm then it scales the results so that the sum of all importances is equal to 1, You can access the resuls using the feature_importances_ variable. FOr edxmple the following code trains a RandomForestClassifier on the iris dataset, and it outputs each features importance. It seem the most important features are the petal length(44%) and the petal width(42%) while sepal length and width are rather unimportant in comparison (11% and 2% respectively).

In [21]:
from sklearn.datasets import load_iris

# Load dataset
iris = load_iris()

rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris["data"], iris["target"])

for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
    print(f"{name}: {score}")

sepal length (cm): 0.09317708956215077
sepal width (cm): 0.02242629312316006
petal length (cm): 0.426598139250034
petal width (cm): 0.4577984780646553


If you train a Random FOrest classifier on the MNIST dataset, and plot each pixels importance, you get the image below.

Random Forests are very useful to get a quick understanding of what features are actually important and matter, in particular this can be helpful for feature selection.

## 4. Boosting